In [1]:
import pandas as pd
import numpy as np
import random
import math
import time

In [2]:
#Ustalenie liczby graczy
#Fixing number of players
n = 25
#Ustalenie liczby iteracji
#Fixing number of iteration
t = 30
#Ustalenie limitu długości życia
#Fixing limit of lifespan
xt = 20

In [3]:
#Tworzenie indeksów
#Create indexes
Gra = np.zeros((n, 1))

index = np.arange(n)
Gra[:,0] = index

#### Możliwe strategie bazowe:
- egoistyczna (0, E) - zawsze zdrada
- kooperatywna (1, K) - współpraca z każdym
- podobnych sobie (2, P) - współpraca z graczami podzielającymi w większości ten sam fenotyp, zdrada innych
- różnych sobie (3, R) - współpraca z graczami o innym fenotypie, zdrada graczy o podobnym fenotypie
- familijna (4, F) - współpraca tylko z własnymi rodzicami i potomkami
- asertywna (5, A) - otwarcie na współpracę z każdym, dopóki liczba punktów nie spadnie poniżej progu krytycznego
#### Possible basic strategies:
- egoistic (0, E) - always the betrayal
- cooperative (1, K) - cooperation with everyone
- similar (2, P) - cooperation with palyers with similar phenotype
- different (3, R) - cooperation with players with different phenotypes
- familiar (4, F) - cooperation only with own parents and descendants
- assertive (5, A) - cooperation with everyone as long as the number of points doesn't fall below the critical threshold

In [4]:
l_s = 6
strategia = np.random.randint(l_s, size=(n, 1))
Gra = np.hstack((Gra, strategia))

In [5]:
print(f"Strategia egoistyczna występuje {np.count_nonzero(Gra[:, 1] == 0)} razy.")
print(f"Strategia kooperacyjna występuje {np.count_nonzero(Gra[:, 1] == 1)} razy.")
print(f"Strategia podobnych sobie występuje {np.count_nonzero(Gra[:, 1] == 2)} razy.")
print(f"Strategia różnych sobie występuje {np.count_nonzero(Gra[:, 1] == 3)} razy.")
print(f"Strategia familijna występuje {np.count_nonzero(Gra[:, 1] == 4)} razy.")
print(f"Strategia asertywna występuje {np.count_nonzero(Gra[:, 1] == 5)} razy.")

Strategia egoistyczna występuje 5 razy.
Strategia kooperacyjna występuje 1 razy.
Strategia podobnych sobie występuje 4 razy.
Strategia różnych sobie występuje 3 razy.
Strategia familijna występuje 5 razy.
Strategia asertywna występuje 7 razy.


#### Możliwa pamięć:
- brak (0, B) - gracz prostępuje zawsze tak samo, niezależnie od poprzednich działań innych graczy
- długa (1, D) - gracz jeżeli zostanie zdradzony, nigdy już nie podejmie współpracy ze zdrajcą
#### Possible memory:
- lack (0, B) - player always acts a same, regardless of a previous actions of other players
- long (1, D) - if a player is betrayed, he will never cooperate with betrayer again

In [6]:
l_p = 2
pamiec = np.random.randint(l_p, size=(n, 1))
Gra = np.hstack((Gra, pamiec))

In [7]:
print(f"Braki pamięci posiada: {np.count_nonzero(Gra[:, 2] == 0)} graczy.")
print(f"Pamięć długą posiada: {np.count_nonzero(Gra[:, 2] == 1)} graczy.")

Braki pamięci posiada: 11 graczy.
Pamięć długą posiada: 14 graczy.


#### Możliwy tryb życia:
- koczowniczy [0] - jednostka zmienia swoje położenie co turę
- zakorzeniony [1] - jednostka nigdy nie zmienia swojego położenia w trakcie gry
- osiadły [2] - jednostka nie zmienia swojego położenia w trakcie gry, chyba że na danym terenie występuje przeludnienie (dla limitu 1000, pop = 700) (w planach)
#### Possible lifestyle:
- nomadic - player changes his location every turn
- rooted - player never changes his location every turn
- settled - player doesn't change his location every turn, unless there is overpopulation in the area (for a limit of 1000, pop = 700) (in the plans)

In [8]:
l_t = 2
tryb_zycia= np.random.randint(l_t, size=(n, 1))
Gra = np.hstack((Gra, tryb_zycia))

In [9]:
print(f"Koczowniczy tryb życia prowadzi: {np.count_nonzero(Gra[:, 3] == 0)} graczy.")
print(f"Zakorzeniony tryb życia prowadzi: {np.count_nonzero(Gra[:, 3] == 1)} graczy.")

Koczowniczy tryb życia prowadzi: 13 graczy.
Zakorzeniony tryb życia prowadzi: 12 graczy.


#### Status początkowy
Wszystkie jednostki rozpoczynają ze 100 punktami, wiekiem 0 oraz statusem żywym (0). Zmniejszenie się liczby punktów poniżej 0 oznacza śmierć (1).
#### Initial status
Every unit starts with 100 points, o age and alive status (0). Decreasing the number of points below 0 means death (1).

In [10]:
pkt = np.full_like(Gra[:, :1], 100)
Gra = np.hstack((Gra, pkt))

In [11]:
wiek = np.full_like(Gra[:, :1], 0)
Gra = np.hstack((Gra, wiek))

In [12]:
status = np.full_like(Gra[:, :1], 0)
Gra = np.hstack((Gra, status))

In [13]:
Fenotyp = np.random.randint(2, size=(n, 1))
Gra = np.hstack((Gra, Fenotyp))

FenotypA = np.full_like(Gra[:, :1], 0)
Gra = np.hstack((Gra, FenotypA))
FenotypA = np.where(Gra[:,7] == 0, 100, Gra[:, 8])
Gra[:,8] = FenotypA

FenotypB = np.full_like(Gra[:, :1], 0)
Gra = np.hstack((Gra, FenotypB))
FenotypB = np.where(Gra[:,7] == 1, 100, Gra[:, 9])
Gra[:,9] = FenotypB

Gra = np.delete(Gra, 7, axis=1)

In [14]:
print(f"Fenotyp A występuje {np.count_nonzero(Gra[:, 7] == 100)} razy.\nFenotyp B występuje {np.count_nonzero(Gra[:, 8] == 100)} razy.")

Fenotyp A występuje 11 razy.
Fenotyp B występuje 14 razy.


#### Lokalizacja
Jednoski zostaną rozrzucone w 4 przestrzeniach oddzielonych barierami naturalnymi od reszty lokalizacji.
#### Areas
Units are scattered in 4 locations separated by narural barriers from the rest.

In [15]:
Lokalizacja = np.random.randint(4, size=(n, 1))
Gra = np.hstack((Gra, Lokalizacja))

In [16]:
print(f"Liczba graczy w danej lokalicjacji: 0: {np.count_nonzero(Gra[:, 9] == 0)}")
print(f"Liczba graczy w danej lokalicjacji: 1: {np.count_nonzero(Gra[:, 9] == 1)}")
print(f"Liczba graczy w danej lokalicjacji: 2: {np.count_nonzero(Gra[:, 9] == 2)}")
print(f"Liczba graczy w danej lokalicjacji: 3: {np.count_nonzero(Gra[:, 9] == 3)}")

Liczba graczy w danej lokalicjacji: 0: 9
Liczba graczy w danej lokalicjacji: 1: 5
Liczba graczy w danej lokalicjacji: 2: 6
Liczba graczy w danej lokalicjacji: 3: 5


#### Powód śmierci
Możliwe wartości: 0 - status żywy, 1 - naturalny, 2 - walka
#### Cause of death
Possible values: 0 - alive status, 1 - natural, 2 - combat

In [17]:
Powód = np.full_like(Gra[:, :1], 0)
Gra = np.hstack((Gra, Powód))

In [18]:
Gra

array([[  0.,   0.,   1.,   1., 100.,   0.,   0.,   0., 100.,   0.,   0.],
       [  1.,   5.,   0.,   0., 100.,   0.,   0.,   0., 100.,   2.,   0.],
       [  2.,   5.,   1.,   1., 100.,   0.,   0., 100.,   0.,   0.,   0.],
       [  3.,   5.,   1.,   1., 100.,   0.,   0.,   0., 100.,   0.,   0.],
       [  4.,   4.,   0.,   0., 100.,   0.,   0.,   0., 100.,   2.,   0.],
       [  5.,   0.,   1.,   1., 100.,   0.,   0., 100.,   0.,   2.,   0.],
       [  6.,   2.,   1.,   0., 100.,   0.,   0., 100.,   0.,   0.,   0.],
       [  7.,   5.,   1.,   0., 100.,   0.,   0., 100.,   0.,   0.,   0.],
       [  8.,   0.,   0.,   1., 100.,   0.,   0., 100.,   0.,   0.,   0.],
       [  9.,   2.,   1.,   0., 100.,   0.,   0.,   0., 100.,   3.,   0.],
       [ 10.,   4.,   0.,   0., 100.,   0.,   0., 100.,   0.,   2.,   0.],
       [ 11.,   0.,   1.,   1., 100.,   0.,   0.,   0., 100.,   2.,   0.],
       [ 12.,   4.,   1.,   1., 100.,   0.,   0.,   0., 100.,   1.,   0.],
       [ 13.,   3.,   0.,

#### Pamięć graczy
W osobnych strukturach są zapisywane informacje zapamiętane przez graczy. Mają oni możliwość zapamiętania zdrajców, rodziców i potomków.
#### Player's memory
In other structures are saved information remembered by players. They have possibility remember betrayers, parents and descendants.

In [19]:
Gra_potomkowie = np.zeros((n, n))
Gra_rodzice = np.zeros((n, n))
Gra_zdrajcy = np.zeros((n, n))

#### Zasady gry
Bazą do zasad gry jest dylemat więźnia. Punktacja jest zależna od malejącej funkcji populacji. Punktacja scenariuszy:
- Obopólna pomoc: obie strony otrzymują po F pkt. (Suma pkt. 2F)
- Obopólna zdrada: obie strony otrzymują po F-11 pkt. (Suma pkt. 2F-22)
- Jednostronna zdrada: strona zdradzająca zyskuje F+10 pkt., strona zdradzona zyskuje traci F-20 pkt. (Suma pkt. 2F-10)
#### Game's rules
Base for the game's rules is prisoner's dilemma. The scoring depends on the decreasing function of the population. Scoring of scenarios:
- Mutual aid: both sides receive F points each. (In total 2F points)
- Mutual betrayal: both sides receive F-11 points each. (In total 2F-22 points)
- Unilateral betrayal: the betrayer gains F+10 points, the betrayed player receives F-20 points. (In total 2F-10 points)


Wzór na funkcję punktów:\
F(Pop) = 10,05-0.0043*Pop-0.00000868*Pop^2+0.000004*Pop^3\
Liczba punktów jest zaokrąglana do dwóch miejsc po przecinku.\
\
Formula for the function of points:\
F(Pop) = 10,05-0.0043*Pop-0.00000868*Pop^2+0.000004*Pop^3\
The number of points is rounded to two decimal places.

In [20]:
#Macierz sąsiedztwa przedstawiająca prawdopodobieństwo zmiany lokalizacji dla koczowników
#Neighborhood matrix showing probability of location change for nomads
Mapa = np.array([[0, 0.5, 0.5, 0],
     [0.5, 0, 0, 0.5],
     [0.5, 0, 0, 0.5],
     [0, 0.5, 0.5, 0]])
#Poniżej macierz sąsiedztwa dla przestrzeni z 9 lokalizacjami
#Bellow neighborhood matrix for area with 9 locations
#Mapa = np.array([[0, 0.5, 0, 0.5, 0, 0, 0, 0, 0],
#                 [1/3, 0, 1/3, 0, 1/3, 0, 0, 0, 0],
#                 [0, 0.5, 0, 0, 0, 0.5, 0, 0, 0],
#                 [1/3, 0, 0, 0, 1/3, 0, 1/3, 0, 0],
#                 [0, 0.25, 0, 0.25, 0, 0.25, 0, 0.25, 0],
#                 [0, 0, 1/3, 0, 1/3, 0, 0, 0, 1/3],
#                 [0, 0, 0, 0.5, 0, 0, 0, 0.5, 0],
#                 [0, 0, 0, 0, 1/3, 0, 1/3, 0, 1/3],
#                 [0, 0, 0, 0, 0, 0.5, 0, 0.5, 0]])
#Pojemność środowiska
#Carrying capacity
Limit=1000
#Tempo wzrostu populacji
#Population growth rate
r_pop=0.2
#Prawdopodobieństwo mutacji
#Mutation probability
m = 0.02

In [21]:
#Funkcja definiująca postępowanie drugiego gracza w oparciu o strategie
#Function that defines the behavior of second player based on strategies
def stg():
    global g2
    #Pamięć B
    if Gra[j, 2] == 0:
        #Stg K
        if Gra[j, 1] == 1:
            g2 = 1
        #Stg E
        elif Gra[j, 1] == 0:
            g2 = 0
        #Stg P
        elif Gra[j, 1] == 2:
            if Gra[j, 7]==Gra[j, 8]:
                g2 = 1
            else:
                N_i2 = np.argmax(Gra[i, 7:9]) + 7
                N_j2 = np.argmax(Gra[j, 7:9]) + 7
                if N_j2 == N_i2:
                    g2 = 1
                else:
                    g2 = 0
        #Stg R
        elif Gra[j, 1] == 3:
            Z_j2 = np.argmax(Gra[i, 7:9]) + 7
            Z_i2 = np.argmax(Gra[j, 7:9]) + 7
            if Z_j2 != Z_i2:
                g2 = 1
            else:
                g2 = 0
        #Str F
        elif Gra[j, 1] == 4:
            if Gra_potomkowie[j, i] == 1 or Gra_rodzice[i, j] == 1:
                g2 = 1
            else:
                g2 = 0
        #Str A
        elif Gra[j, 1] == 5:
            pop_pkt = np.count_nonzero((Gra[:, 9] == Gra[j, 9]) & (Gra[:, 6] == 0))
            f_pkt = round(10.05-0.0043*pop_pkt-0.0000175*pop_pkt**2+0.000004*pop_pkt**3, 2)
            if Gra[j, 4] > (f_pkt-20)*3:
                g2 = 1
            else:
                g2 = 0
    #Pamięć D
    elif Gra[j, 2] == 1:
        if Gra_zdrajcy[j,i] == 1:
            g2=0
        else:
            #Stg K
            if Gra[j, 1] == 1:
                g2 = 1
            #Stg E
            elif Gra[j, 1] == 0:
                g2 = 0
            #Stg P
            elif Gra[j, 1] == 2:
                if Gra[j, 7]==Gra[j, 8]:
                    g2 = 1
                else:
                    N_i2 = np.argmax(Gra[i, 7:9]) + 7
                    N_j2 = np.argmax(Gra[j, 7:9]) + 7
                    if N_j2 == N_i2:
                        g2 = 1
                    else:
                        g2 = 0
            #Stg R
            elif Gra[j, 1] == 3:
                Z_j2 = np.argmax(Gra[i, 7:9]) + 7
                Z_i2 = np.argmax(Gra[j, 7:9]) + 7
                if Z_j2 != Z_i2:
                    g2 = 1
                else:
                    g2 = 0
            #Str F
            elif Gra[j, 1] == 4:
                if Gra_potomkowie[j, i] == 1 or Gra_rodzice[i, j] == 1:
                    g2 = 1
                else:
                    g2 = 0
            #Str A
            elif Gra[j, 1] == 5:
                pop_pkt = np.count_nonzero((Gra[:, 9] == Gra[j, 9]) & (Gra[:, 6] == 0))
                f_pkt = round(10.05-0.0043*pop_pkt-0.0000175*pop_pkt**2+0.000004*pop_pkt**3, 2)
                if Gra[j, 4] > (f_pkt-20)*3:
                    g2 = 1
                else:
                    g2 = 0

In [22]:
Lista_wyniki = []
start_time = time.time()
Wyniki_całosc = pd.DataFrame(Gra)
Wyniki_całosc["Runda"] = 0
global Wyniki_r, Lista_wyniki, Gra
for _ in range(t):
    for i in range(0, len(Gra)):
        #Sprawdzenie czy gracz i nie jest martwy
        #Checking if player i isn't dead
        if Gra[i, 6] == 1:
            continue
        for j in range(0, i):
            #Sprawdzenie czy gracz j nie jest martwy
            #Checking if player j isn't dead
            if Gra[j, 6] == 1:
                continue
            #Upewnienie się czy gracz i w trakcie rozgrywki nie stał się martwy
            #Ensure that player i didn't become dead during gameplay
            if Gra[i, 6] == 1:
                continue
            #Sprawdzanie zgodności lokalizacji
            #Check location compatibility
            if Gra[i, 9] != Gra[j, 9]:
                continue
            #Pamięć B
            #Lack memory
            if Gra[i, 2] == 0:
                #Stg K
                if Gra[i, 1] == 1:
                    g1 = 1
                    stg()
                #Stg E
                elif Gra[i, 1] == 0:
                    g1 = 0
                    stg()
                #Stg P
                elif Gra[i, 1] == 2:
                    if Gra[i, 7]==Gra[i, 8]:
                        g1 = 1
                    else:
                        N_i1 = np.argmax(Gra[i, 7:9]) + 7
                        N_j1 = np.argmax(Gra[j, 7:9]) + 7
                        if N_j1 == N_i1:
                            g1 = 1
                        else:
                            g1 = 0
                    stg()
                #Stg R
                elif Gra[i, 1] == 3:
                    Z_j1 = np.argmax(Gra[i, 7:9]) + 7
                    Z_i1 = np.argmax(Gra[j, 7:9]) + 7
                    if Z_j1 != Z_i1:
                        g1 = 1
                    else:
                        g1 = 0
                    stg()
                #Stg F
                elif Gra[i, 1] == 4:
                    if Gra_potomkowie[i, j] == 1 or Gra_rodzice[j, i] == 1:
                        g1 = 1
                    else:
                        g1 = 0
                    stg()
                #Str A
                elif Gra[i, 1] == 5:
                    pop_pkt = np.count_nonzero((Gra[:, 9] == Gra[i, 9]) & (Gra[:, 6] == 0))
                    f_pkt = round(10.05-0.0043*pop_pkt-0.0000175*pop_pkt**2+0.000004*pop_pkt**3, 2)
                    if Gra[i, 4] > (f_pkt-20)*3:
                        g1 = 1
                    else:
                        g1 = 0
                    stg()
            #Pamięć D
            #Long memory
            elif Gra[i, 2] == 1:
                if Gra_zdrajcy[i,j] == 1:
                    g1=0
                    stg()
                else:
                    #Stg K
                    if Gra[i, 1] == 1:
                        g1 = 1
                        stg()
                    #Stg E
                    elif Gra[i, 1] == 0:
                        g1 = 0
                        stg()
                    #Stg P
                    elif Gra[i, 1] == 2:
                        if Gra[i, 7]==Gra[i, 8]:
                            g1 = 1
                        else:
                            N_i1 = np.argmax(Gra[i, 7:9]) + 7
                            N_j1 = np.argmax(Gra[j, 7:9]) + 7
                            if N_j1 == N_i1:
                                g1 = 1
                            else:
                                g1 = 0
                        stg()
                    #Stg R
                    elif Gra[i, 1] == 3:
                        Z_j1 = np.argmax(Gra[i, 7:9]) + 7
                        Z_i1 = np.argmax(Gra[j, 7:9]) + 7
                        if Z_j1 != Z_i1:
                            g1 = 1
                        else:
                            g1 = 0
                        stg()
                    #Stg F
                    elif Gra[i, 1] == 4:
                        if Gra_potomkowie[i, j] == 1 or Gra_rodzice[j, i] == 1:
                            g1 = 1
                        else:
                            g1 = 0
                        stg()
                    #Stg A
                    elif Gra[i, 1] == 5:
                        pop_pkt = np.count_nonzero((Gra[:, 9] == Gra[i, 9]) & (Gra[:, 6] == 0))
                        f_pkt = round(10.05-0.0043*pop_pkt-0.0000175*pop_pkt**2+0.000004*pop_pkt**3, 2)
                        if Gra[i, 4] > (f_pkt-20)*3:
                            g1 = 1
                        else:
                            g1 = 0
                        stg()
            #Wzór na funkcję punktów
            #Points fomula
            pop_pkt = np.count_nonzero((Gra[:, 9] == Gra[i, 9]) & (Gra[:, 6] == 0))
            f_pkt = round(10.05-0.0043*pop_pkt-0.0000175*pop_pkt**2+0.000004*pop_pkt**3, 2)
            #Przyznanie punktów
            #Awarding of points
            if g1 == 1:
                if g2 == 1:
                    Gra[i, 4] += f_pkt
                    Gra[j, 4] += f_pkt
                else:
                    Gra[i, 4] += f_pkt-20
                    Gra[j, 4] += f_pkt+10
            else:
                if g2 == 1:
                    Gra[i, 4] += f_pkt+10
                    Gra[j, 4] += f_pkt-20
                else:
                    Gra[i, 4] += f_pkt-11
                    Gra[j, 4] += f_pkt-11
            if Gra[i, 4] <= 0:
                Gra[i, 6] = 1
                Gra[i, 10] = 2
            if Gra[j, 4] <= 0:
                Gra[j, 6] = 1
                Gra[j, 10] = 2
            #Przypisanie statusu zdrajcy
            #Assignment of betrayer status
            if g2 == 0:
                if Gra_zdrajcy[i,j] == 1:
                    continue
                else:
                    Gra_zdrajcy[i,j] = 1
            if g1 == 0:
                if Gra_zdrajcy[j,i] == 1:
                    continue
                else:
                    Gra_zdrajcy[j,i] = 1
            #Wprowadzenie zachowań koczowniczych gracz j
            #Introducing nomadic lifestyle pleyers j
            if Gra[j, 3] == 0:
                Gra[j, 9] = np.random.choice(np.arange(0, 4), p=Mapa[int(Gra[j,9]), :])
        #Wprowadzenie zachowań koczowniczych gracz i
        #Introducing nomadic lifestyle pleyers i
        if Gra[i, 3] == 0:
            Gra[i, 9] = np.random.choice(np.arange(0, 4), p=Mapa[int(Gra[i, 9]), :])
    print(f"Runda {_+1}")
    #Wprowadzenie wieku
    #Introduction of age
    Gra[Gra[:,6] == 0, 5] += 1
    #Zmiana statusu
    #Changing status
    Gra[Gra[:, 5] == xt, 6] = 1
    Gra[Gra[:, 5] == xt, 10] = 1
    #Wprowadzenie dziedziczenia
    #Introduction of inheritance
    for i in range(len(Gra)):
        if Gra[i,6] == 1 and Gra[i,10] == 1 and Gra[i,4] > 0:
            potomkowie = np.where(Gra_potomkowie[i] == 1)[0]
            liczba_potomków = np.count_nonzero(Gra_potomkowie[i] == 1)
            if liczba_potomków > 0:
                punkty = Gra[i,4]/liczba_potomków
                for potomek in potomkowie:
                    Gra[int(potomek), 4] += punkty
            Gra[i, 4] = 0
    #Wprowadzenie reprodukcji
    #Introduction of reproduction
    for q in range(0, 4):
        N = Gra[Gra[:, 9] == q, :]
        N = len(N[:, 4] > 100)
        #Zaokrąglenie wyniku funkcji populacji w górę
        #Rounding up the result of the population function
        Nd = math.ceil(N*r_pop*(1-(N/Limit)))
        Pop_r = Nd*2
        Kandydaci = Gra[Gra[:,9] == q, :]
        Kandydaci = Kandydaci[Kandydaci[:,4] > 100,:]
        try:
            #Losowanie określonej liczby kandydatów, prawdopodobieństwo wylosowania danego gracza jest ważone liczbą posiadanych punktów
            #Drawing a certain number of candidates, the probability of drawing a given player is weighted by his number of points
            prawdopodobieństwo = Kandydaci[:,4]/np.sum(Kandydaci[:,4])
            indeksy = np.random.choice(Kandydaci[:, 0], size=Pop_r, replace=False, p=prawdopodobieństwo)
            for q2 in range(0, len(indeksy), 2):
                nowy_index = int(max(Gra[:,0]))+1
                nowa_strategia = np.random.choice([Gra[int(indeksy[q2]), 1],
                                                   Gra[int(indeksy[q2+1]), 1],
                                                   np.random.randint(l_s)],
                                                  p = [(1-m)/2, (1-m)/2, m])                  
                nowa_pamięć = np.random.choice([Gra[int(indeksy[q2]), 2],
                                                Gra[int(indeksy[q2+1]), 2],
                                                np.random.randint(l_p)],
                                                p = [(1-m)/2, (1-m)/2, m])
                nowy_tż = np.random.choice([Gra[int(indeksy[q2]), 3],
                                            Gra[int(indeksy[q2+1]), 3],
                                            np.random.randint(l_t)],
                                            p = [(1-m)/2, (1-m)/2, m])              
                Gra[int(indeksy[q2]), 4] = Gra[int(indeksy[q2]), 4] - 50                
                Gra[int(indeksy[q2+1]), 4] = Gra[int(indeksy[q2+1]), 4] - 50                
                nowy_fenotyp_a = (Gra[int(indeksy[q2]), 7] + Gra[int(indeksy[q2+1]), 7]) / 2                
                nowy_fenotyp_b = (Gra[int(indeksy[q2]), 8] + Gra[int(indeksy[q2+1]), 8]) / 2                
                nowa_lokalizacja = Gra[int(indeksy[q2]), 9]                
                Gra = np.append(Gra,
                                np.array([[nowy_index,
                                                nowa_strategia,
                                                nowa_pamięć,
                                                nowy_tż,
                                                100,
                                                0,
                                                0,
                                                nowy_fenotyp_a,
                                                nowy_fenotyp_b,
                                                nowa_lokalizacja,
                                                0]]),
                                axis=0)
                Gra_zdrajcy = np.append(Gra_zdrajcy,
                                        np.zeros((1, Gra_zdrajcy.shape[1])),
                                        axis = 0)
                Gra_zdrajcy = np.append(Gra_zdrajcy,
                                        np.zeros((Gra_zdrajcy.shape[0], 1)),
                                        axis = 1)
                
                Gra_rodzice = np.append(Gra_rodzice,
                                        np.zeros((1, Gra_rodzice.shape[1])),
                                        axis = 0)
                Gra_rodzice = np.append(Gra_rodzice,
                                        np.zeros((Gra_rodzice.shape[0], 1)),
                                        axis = 1)
                
                Gra_potomkowie = np.append(Gra_potomkowie,
                                        np.zeros((1, Gra_potomkowie.shape[1])),
                                        axis = 0)
                Gra_potomkowie = np.append(Gra_potomkowie,
                                        np.zeros((Gra_potomkowie.shape[0], 1)),
                                        axis = 1)
                Gra_rodzice[nowy_index, int(indeksy[q2])] = 1
                Gra_rodzice[nowy_index, int(indeksy[q2+1])] = 1
                
                Gra_potomkowie[int(indeksy[q2]), nowy_index] = 1
                Gra_potomkowie[int(indeksy[q2+1]), nowy_index] = 1
        except:
            continue
    #Zapisanie wyników cząskowych
    #Saving results of the turn data
    Wyniki_częsc = pd.DataFrame(Gra)
    Wyniki_częsc["Runda"] = _+1
    #Tworzenie zbioru wyników całej gry
    Wyniki_całosc = pd.concat([Wyniki_całosc, Wyniki_częsc], ignore_index=True)
Wyniki_całosc.columns = ["Indeks", "Strategia bazowa", "Pamięć", "Tryb życia", "Punkty",
                         "Wiek", "Status", "Fenotyp A", "Fenotyp B", "Lokalizacja",
                         "Powód śmierci", "Runda"]
execution_time = time.time() - start_time
print(f"Czas wykonania pętli: {round(execution_time, 2)} sekundy")

Runda 1
Runda 2
Runda 3
Runda 4
Runda 5
Runda 6
Runda 7
Runda 8
Runda 9
Runda 10
Runda 11


Runda 12
Runda 13
Runda 14
Runda 15
Runda 16
Runda 17
Runda 18
Runda 19
Runda 20
Runda 21
Runda 22
Runda 23
Runda 24
Runda 25
Runda 26
Runda 27
Runda 28
Runda 29
Runda 30
Czas wykonania pętli: 283.71 sekundy


In [23]:
Wyniki_całosc

,Indeks,Strategia bazowa,Pamięć,Tryb życia,Punkty,Wiek,Status,Fenotyp A,Fenotyp B,Lokalizacja,Powód śmierci,Runda
0,0.0,0.0,1.0,1.0,147.10,1.0,0.0,0.000000,100.000000,0.0,0.0,0
1,1.0,5.0,0.0,0.0,130.10,1.0,0.0,0.000000,100.000000,2.0,0.0,0
2,2.0,5.0,1.0,1.0,110.11,1.0,0.0,100.000000,0.000000,0.0,0.0,0
3,3.0,5.0,1.0,1.0,100.10,1.0,0.0,0.000000,100.000000,0.0,0.0,0
4,4.0,4.0,0.0,0.0,177.11,1.0,0.0,0.000000,100.000000,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
14861,1562.0,0.0,0.0,0.0,100.00,0.0,0.0,36.230469,63.769531,2.0,0.0,30
14862,1563.0,4.0,1.0,0.0,100.00,0.0,0.0,39.843750,60.156250,2.0,0.0,30
14863,1564.0,4.0,0.0,0.0,100.00,0.0,0.0,36.621094,63.378906,2.0,0.0,30
14864,1565.0,0.0,1.0,0.0,100.00,0.0,0.0,34.765625,65.234375,2.0,0.0,30


In [24]:
#Zamiana typów danych i oznaczeń na potrzeby analizy wyników
#Converting data types and designations for performance analysis
Wyniki_całosc["Indeks"] = Wyniki_całosc["Indeks"].astype(int)
Wyniki_całosc["Strategia bazowa"] = Wyniki_całosc["Strategia bazowa"].replace(0, "E")
Wyniki_całosc["Strategia bazowa"] = Wyniki_całosc["Strategia bazowa"].replace(1, "K")
Wyniki_całosc["Strategia bazowa"] = Wyniki_całosc["Strategia bazowa"].replace(2, "P")
Wyniki_całosc["Strategia bazowa"] = Wyniki_całosc["Strategia bazowa"].replace(3, "R")
Wyniki_całosc["Strategia bazowa"] = Wyniki_całosc["Strategia bazowa"].replace(4, "F")
Wyniki_całosc["Strategia bazowa"] = Wyniki_całosc["Strategia bazowa"].replace(5, "A")
Wyniki_całosc["Pamięć"] = Wyniki_całosc["Pamięć"].replace(0, "B")
Wyniki_całosc["Pamięć"] = Wyniki_całosc["Pamięć"].replace(1, "D")
Wyniki_całosc["Tryb życia"] = Wyniki_całosc["Tryb życia"].replace(0, "K")
Wyniki_całosc["Tryb życia"] = Wyniki_całosc["Tryb życia"].replace(1, "Z")
Wyniki_całosc["Wiek"] = Wyniki_całosc["Wiek"].astype(int)
Wyniki_całosc["Status"] = Wyniki_całosc["Status"].replace(0, "Ż")
Wyniki_całosc["Status"] = Wyniki_całosc["Status"].replace(1, "Ś")
Wyniki_całosc["Lokalizacja"] = Wyniki_całosc["Lokalizacja"].astype(int)
Wyniki_całosc["Powód śmierci"] = Wyniki_całosc["Powód śmierci"].replace(0, "")
Wyniki_całosc["Powód śmierci"] = Wyniki_całosc["Powód śmierci"].replace(1, "N")
Wyniki_całosc["Powód śmierci"] = Wyniki_całosc["Powód śmierci"].replace(2, "W")
Wyniki_całosc["Strategia synkretyczna"] = Wyniki_całosc["Strategia bazowa"] + Wyniki_całosc["Pamięć"] + Wyniki_całosc["Tryb życia"]
Wyniki_całosc

,Indeks,Strategia bazowa,Pamięć,Tryb życia,Punkty,Wiek,Status,Fenotyp A,Fenotyp B,Lokalizacja,Powód śmierci,Runda,Strategia synkretyczna
0,0,E,D,Z,147.10,1,Ż,0.000000,100.000000,0,,0,EDZ
1,1,A,B,K,130.10,1,Ż,0.000000,100.000000,2,,0,ABK
2,2,A,D,Z,110.11,1,Ż,100.000000,0.000000,0,,0,ADZ
3,3,A,D,Z,100.10,1,Ż,0.000000,100.000000,0,,0,ADZ
4,4,F,B,K,177.11,1,Ż,0.000000,100.000000,0,,0,FBK
...,...,...,...,...,...,...,...,...,...,...,...,...,...
14861,1562,E,B,K,100.00,0,Ż,36.230469,63.769531,2,,30,EBK
14862,1563,F,D,K,100.00,0,Ż,39.843750,60.156250,2,,30,FDK
14863,1564,F,B,K,100.00,0,Ż,36.621094,63.378906,2,,30,FBK
14864,1565,E,D,K,100.00,0,Ż,34.765625,65.234375,2,,30,EDK


In [25]:
#Zapisanie wyników końcowych w osobnej tabeli dla ułatwienia analizy wyników
#Saving final results in a separate table for easy analysis of results
Wyniki_k = pd.DataFrame(Gra[:, 1:],
                        index = Gra[:,0].astype(int),
                        columns = ["Strategia bazowa",
                                       "Pamięć",
                                       "Tryb życia",
                                       "Punkty",
                                       "Wiek",
                                       "Status",
                                       "Fenotyp A",
                                       "Fenotyp B",
                                       "Lokalizacja",
                                       "Powód śmierci"])

In [26]:
#Zamiana typów danych i oznaczeń na potrzeby analizy wyników
#Converting data types and designations for performance analysis
Wyniki_k["Strategia bazowa"] = Wyniki_k["Strategia bazowa"].replace(0, "E")
Wyniki_k["Strategia bazowa"] = Wyniki_k["Strategia bazowa"].replace(1, "K")
Wyniki_k["Strategia bazowa"] = Wyniki_k["Strategia bazowa"].replace(2, "P")
Wyniki_k["Strategia bazowa"] = Wyniki_k["Strategia bazowa"].replace(3, "R")
Wyniki_k["Strategia bazowa"] = Wyniki_k["Strategia bazowa"].replace(4, "F")
Wyniki_k["Strategia bazowa"] = Wyniki_k["Strategia bazowa"].replace(5, "A")
Wyniki_k["Pamięć"] = Wyniki_k["Pamięć"].replace(0, "B")
Wyniki_k["Pamięć"] = Wyniki_k["Pamięć"].replace(1, "D")
Wyniki_k["Tryb życia"] = Wyniki_k["Tryb życia"].replace(0, "K")
Wyniki_k["Tryb życia"] = Wyniki_k["Tryb życia"].replace(1, "Z")
Wyniki_k["Wiek"] = Wyniki_k["Wiek"].astype(int)
Wyniki_k["Status"] = Wyniki_k["Status"].replace(0, "Ż")
Wyniki_k["Status"] = Wyniki_k["Status"].replace(1, "Ś")
Wyniki_k["Lokalizacja"] = Wyniki_k["Lokalizacja"].astype(int)
Wyniki_k["Powód śmierci"] = Wyniki_k["Powód śmierci"].replace(0, "")
Wyniki_k["Powód śmierci"] = Wyniki_k["Powód śmierci"].replace(1, "N")
Wyniki_k["Powód śmierci"] = Wyniki_k["Powód śmierci"].replace(2, "W")
Wyniki_k["Strategia synkretyczna"] = Wyniki_k["Strategia bazowa"] + Wyniki_k["Pamięć"] + Wyniki_k["Tryb życia"]
Wyniki_k

,Strategia bazowa,Pamięć,Tryb życia,Punkty,Wiek,Status,Fenotyp A,Fenotyp B,Lokalizacja,Powód śmierci,Strategia synkretyczna
0,E,D,Z,0.00,20,Ś,0.000000,100.000000,0,N,EDZ
1,A,B,K,-9.78,4,Ś,0.000000,100.000000,1,W,ABK
2,A,D,Z,0.00,20,Ś,100.000000,0.000000,0,N,ADZ
3,A,D,Z,0.00,20,Ś,0.000000,100.000000,0,N,ADZ
4,F,B,K,-2.07,17,Ś,0.000000,100.000000,2,W,FBK
...,...,...,...,...,...,...,...,...,...,...,...
1562,E,B,K,100.00,0,Ż,36.230469,63.769531,2,,EBK
1563,F,D,K,100.00,0,Ż,39.843750,60.156250,2,,FDK
1564,F,B,K,100.00,0,Ż,36.621094,63.378906,2,,FBK
1565,E,D,K,100.00,0,Ż,34.765625,65.234375,2,,EDK


In [27]:
#Integracja dodatkowych tabel będących strukturami pamięci z wynikami końcowymi
#Integration of additional tables which are memory structures with final results
test1 = pd.DataFrame(Gra_potomkowie)
Wyniki_k["Potomkowie"] = test1.apply(lambda row: np.where(row == 1)[0], axis=1)
Wyniki_k["Potomkowie"] = Wyniki_k["Potomkowie"].apply(lambda x: str(x).replace("[", "").replace("]", ""))
test2 = pd.DataFrame(Gra_rodzice)
Wyniki_k["Rodzice"] = test2.apply(lambda row: np.where(row == 1)[0], axis=1)
Wyniki_k["Rodzice"] = Wyniki_k["Rodzice"].apply(lambda x: str(x).replace("[", "").replace("]", ""))
test3 = pd.DataFrame(Gra_zdrajcy)
Wyniki_k["Zdrajcy"] = test3.apply(lambda row: np.where(row == 1)[0], axis=1)
Wyniki_k["Zdrajcy"] = Wyniki_k["Zdrajcy"].apply(lambda x: str(x).replace("[", "").replace("]", ""))
Wyniki_k

,Strategia bazowa,Pamięć,Tryb życia,Punkty,Wiek,Status,Fenotyp A,Fenotyp B,Lokalizacja,Powód śmierci,Strategia synkretyczna,Potomkowie,Rodzice,Zdrajcy
0,E,D,Z,0.00,20,Ś,0.000000,100.000000,0,N,EDZ,25 40 47 58 64 79 117 135 170 293 337,,4 6 8 10 18 23 25 29 30 31 32 3...
1,A,B,K,-9.78,4,Ś,0.000000,100.000000,1,W,ABK,29,,4 5 8 12 15 23 24 25 28 29 30 31 35 37
2,A,D,Z,0.00,20,Ś,100.000000,0.000000,0,N,ADZ,33 38 93 111 128 189 217 279 328 389,,0 4 8 10 17 18 21 22 23 25 29 3...
3,A,D,Z,0.00,20,Ś,0.000000,100.000000,0,N,ADZ,40 48 55 63 80 95 126 153 164 196 260,,0 4 6 8 10 18 23 25 29 30 31 3...
4,F,B,K,-2.07,17,Ś,0.000000,100.000000,2,W,FBK,37 44 54 74 90 104,,0 5 6 8 11 12 15 18 23 24 28 2...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1562,E,B,K,100.00,0,Ż,36.230469,63.769531,2,,EBK,,398 686,
1563,F,D,K,100.00,0,Ż,39.843750,60.156250,2,,FDK,,397 811,
1564,F,B,K,100.00,0,Ż,36.621094,63.378906,2,,FBK,,309 1083,
1565,E,D,K,100.00,0,Ż,34.765625,65.234375,2,,EDK,,613 716,


In [28]:
#Zapisanie wyników symulacji jako plik Excela
#Saving simulation results as Excel file
Wyniki_badania = pd.ExcelWriter("Wyniki_badania.xlsx")
Wyniki_całosc.to_excel(Wyniki_badania, sheet_name="Wyniki szczegółowe")
Wyniki_k.to_excel(Wyniki_badania, sheet_name="Wyniki końcowe")
Wyniki_badania.save()